In [ ]:
from datasets import load_dataset
ds = load_dataset("deepmind/narrativeqa")

In [2]:
data = load_dataset('deepmind/narrativeqa', split='train')

In [10]:
prompt="You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:"

In [ ]:
data[300]['document']['summary']['title']

In [11]:
lb = load_dataset('THUDM/LongBench', "narrativeqa", split='train')

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [5]:
mode_path = '/mnt/xiyu/Qwen/Qwen2___5-7B'
tokenizer = AutoTokenizer.from_pretrained(mode_path, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from tqdm import tqdm
import json
cnt=0
all_length=0
max_length=128000
paper_ids=[]
papers={}
for json_obj in tqdm(data):
    unique_id=json_obj['document']['id']
    if unique_id not in paper_ids:
        paper_ids.append(unique_id)
        tokenized_paper=tokenizer(json_obj['document']['text'], truncation=False, return_tensors="pt").input_ids[0]
        if len(tokenized_paper) > max_length:
            half = int(max_length/2)
            new_paper = tokenizer.decode(tokenized_paper[:half], skip_special_tokens=True)+tokenizer.decode(tokenized_paper[-half:], skip_special_tokens=True)
        else:
            new_paper=json_obj['document']['text']
        papers[unique_id]=new_paper
with open(f'dataset/narrativeqa_tokenized.json', "w", encoding="utf-8") as f:
    json.dump(papers, f, ensure_ascii=False)

In [7]:
from tqdm import tqdm
import json
paper_dict={}
with open(f'dataset/narrativeqa_tokenized.json', "r", encoding="utf-8") as f:
    paper_dict=json.load(f)

In [ ]:
train_dataset=[]
prompt_format="You are given a story, which can be either a novel or a movie script, and a question. Answer the question asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nStory: {context}\n\nNow, answer the question based on the story asconcisely as you can, using a single phrase if possible. Do not provide any explanation.\n\nQuestion: {input}\n\nAnswer:"
for json_obj in tqdm(data):
    unique_id=json_obj['document']['id']
    temp_dict={"context":paper_dict[unique_id],"input":json_obj['question']['text']}
    prompt=prompt_format.format(**temp_dict)
    answer=json_obj['answers'][0]['text']
    train_dataset.append({
    "instruction": prompt,
    "output": answer,
    "id": unique_id
    })
    with open("/mnt/xiyu/dataset/narrativeqa_train.jsonl", "a", encoding="utf-8") as f:
        json.dump({"instruction": prompt,"output": answer,"id": unique_id}, f, ensure_ascii=False)
        f.write('\n')

In [16]:
with open('/mnt/qwen2.5-72B/dataset/narrativeqa_train.json', "r", encoding="utf-8") as f:
    paper_dict=json.load(f)